### Difference between ID3 and C4.5:
* ID3 use information gain while C4.5 use information gain ratio
* Information gain is depending on training data, does not have "absolute" meaning. While use information gain ratio we could solve this.
* $g_r(D,A) = \frac{g(D,A)}{SplitInfo(D,A)}$ 
* $SplitInfo(D,A) = -\sum_{j=1}^{J} \frac{|S_j|}{|S|} log\frac{|S_j|}{|S|}$, where J is splitting the dataset into J partitions.
* 样本量巨大，其Splite_info会趋向无穷大，则gain_ratio趋向无穷小  
* 当存在|Di| ≈ |D|时，split_info将非常小（可能等于0），从而导致增益比例异常大，改进方法是计算每个属性的信息增益，对于超过平均信息增益的属性再进一步根据增益比例来选取属性。

In [171]:
import time
import logging
import numpy as np
import pandas as pd


from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score


In [172]:
total_class = 10

In [173]:
def log(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        logging.debug('start %s()' % func.__name__)
        ret = func(*args, **kwargs)
        
        end_time= time.time()
        logging.debug('end %s(), cost %s seconds' % (func.__name__, end_time-start_time))
        
        return ret
    return wrapper

In [174]:
if __name__ == '__main__':
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    
    raw_data = pd.read_csv('/Users/guozhiqi-seven/Documents/Statistical Learning/Decision Tree/train.csv',header=0) 
    data = raw_data.values
    
    img = data[:,1:]
    labels = data[:,0]   

In [175]:
#直接用二值化之后的dataset (cv2 under python3 is available)
features = np.loadtxt('features.out')

In [176]:
features.shape

(42000, 784)

In [177]:
features = features[:500]
labels = labels[:500]

In [178]:
#2/3 Training set
#1/3 Testing  set
train_features,test_features,train_labels,test_labels = train_test_split(features,labels,test_size=0.33)

In [179]:
class Tree(object):
    def __init__(self,node_type,Class=None,feature=None):
        self.node_type = node_type
        self.Class = Class
        self.feature = feature
        self.dict = {}  #不需要构造新的数据类型来存储决策树，使用字典dict即可方便的存储节点信息
    def add_tree(self,val,tree):
        self.dict[val]=tree
    def predict(self,features):
        if self.node_type == 'leaf':
            return self.Class
        tree = self.dict[features[self.feature]]
        
        return tree.predict(features)       

In [180]:
def calc_entropy(x):
    '''
    calculate entropy
    x: labels
    '''
    
    #H(X) = -sum(p_i * log(p_i))
    x_value_list = set([x[i] for i in range(x.shape[0])])
    entropy = 0.0
    
    for x_value in x_value_list:
        p = float(x[x==x_value].shape[0]) / x.shape[0]
        log_p = np.log2(p)
        
        entropy -= p*log_p
    
    return entropy #return啊你倒是return啊你妹...

In [181]:
def calc_condition_entropy(x,y):
    '''
    conditional entropy H(y|x)
    
    x:feature
    y:label
    '''
    x_value_list = set([x[i] for i in range(x.shape[0])])
    entropy = 0.0
    
    for x_value in x_value_list:
        sub_y = y[x==x_value]
        temp_entropy = calc_entropy(sub_y)
        entropy += (float(sub_y.shape[0]) / y.shape[0]) * temp_entropy
        #print float(sub_y.shape[0]) / y.shape[0]
    
    return entropy #return啊你倒是return啊你妹...
       

In [182]:
def split_info(x,y):
    '''
    split information/ intrinsic information
    x:feature
    y:label
    '''
    x_value_list = set([x[i] for i in range(x.shape[0])])
    split_info = 0.0
    
    for x_value in x_value_list:
        sub_y = y[x==x_value]
        split_info += float(sub_y.shape[0])/y.shape[0] * np.log2(float(sub_y.shape[0]/y.shape[0]))
        #print np.log2(float(sub_y.shape[0])/y.shape[0])
    return -split_info   

In [183]:
def calc_entropy_g(x,y):
    '''
    g(D,A) = H(D) - H(D|A)
    '''
    base_entropy = calc_entropy(y)
    condition_entropy = calc_condition_entropy(x,y)
    
    entropy_g = base_entropy - condition_entropy
    
    return entropy_g

In [184]:
@log
def train(train_set,train_label,features,epsilon):
    return recurse_train(train_set,train_label,features,epsilon) 

In [185]:
def recurse_train(train_set,train_label,features,epsilon):
    global total_class
    
    LEAF = 'leaf'
    INTERNAL = 'internal'
    
    #Step1: 如果train_set所有实例属于同一类C_k
    label_set = set(train_label)
    
    if len(label_set) == 1:
        return Tree(LEAF,Class = label_set.pop())
    
    #Step2: 如果feature为空
    (max_class,max_len)=max([(i,len(filter(lambda x: x==i, train_label))) for i in xrange(total_class)],
                            key=lambda x:x[1])
    if len(features) == 0:
        return Tree(LEAF, Class = max_class)
    
    #Step3: calculate entropy
    max_feature = 0
    max_gda_ratio = 0 #g_r(D,A)
    
    D = train_label
    H_D = calc_entropy(D)
    
    for feature in features:
        #A = np.array(features[:,feature].flat)
        A = np.array(train_set[:,feature].flat)
        gda = H_D - calc_condition_entropy(A,D)
        
        split_information = split_info(A,D)
        #print split_information
        if split_information == 0:
            continue
        gda_ratio = gda/split_info(A,D)   #信息增益比
        #print gda_ratio
        
        if gda_ratio > max_gda_ratio:
            max_gda_ratio = gda_ratio
            max_feature = feature
    
    #Step4:entropy 小于阈值的情况
    if max_gda_ratio < epsilon:
        return Tree(LEAF,Class = max_class)
    
    #Step5:构建非空子集
    sub_features = filter(lambda x:x!= max_feature, features)
    tree = Tree(INTERNAL, feature=max_feature)
    
    feature_col = np.array(train_set[:,max_feature].flat) #max feature
    features_value_list = set([feature_col[i] for i in range(feature_col.shape[0])]) #信息增益最大特征A_g的每一可能值a_i
    
    for feature_value in features_value_list:
        index = []
        for i in xrange(len(train_label)):
            if train_set[i][max_feature] == feature_value:
                index.append(i)
        
        sub_train_set= train_set[index]
        sub_train_label = train_label[index]
        
        sub_tree = recurse_train(sub_train_set,sub_train_label,sub_features,epsilon)
        tree.add_tree(feature_value,sub_tree)
        
    return tree  
    

In [186]:
@log
def predict(test_set,tree):
    result = []
    
    for feature in test_set:
        temp_prediction = tree.predict(feature)
        result.append(temp_prediction)
    
    return np.array(result)

Did not see much difference, probably since dataset do not have many labels. Also features are pretty much 'discrete'.

In [187]:
tree = train(train_features,train_labels,[i for i in range(784)],0.01)
test_predict = predict(test_features,tree)
score = accuracy_score(test_labels,test_predict)

print "The accruacy socre is ", score

DEBUG:root:start train()
/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: divide by zero encountered in log2
DEBUG:root:end train(), cost 0.456895828247 seconds
DEBUG:root:start predict()
DEBUG:root:end predict(), cost 0.00173211097717 seconds


The accruacy socre is  0.121212121212
